In [44]:
import geopandas as gpd
import os
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pickle
import copy 

warnings.filterwarnings('ignore')
tqdm.pandas()

# Construct Roads Graph

## Load Data

In [45]:
TEST = False

In [46]:
MAPA_PATH = "../../../data/Mapa/test_data/center_tlv/" if TEST else "../../../data/Mapa/Ben Gurion University/Gisrael/"
OUTPUT_PATH = "../../output_data/roads/test/center_tlv/" if TEST else "../../output_data/roads/"


In [47]:
roads_df = gpd.read_file(MAPA_PATH + 'is_str.shp')

In [48]:
roads_df.shape

(647504, 38)

## Basic Edges

In [49]:
shape_edges_df = roads_df[['UserID', 'ObjID', 'FJunction', 'TJunction', 'RoadType', 
                     'OneWay', 'FromSpeedL', 'ToSpeedL', 
                     'AprxSpeedL', 'Seconds', 'Length', 'Roaddirect', 'DirectionM', 'F_ZLev', 'T_ZLev']]
shape_edges_df = shape_edges_df.reset_index()

In [50]:
edges_dict = dict()
adjacency_list = dict()
number_of_two_way = 0
number_of_ft = 0
number_of_tf = 0

def create_edges_dict(e):
    global edges_dict
    global number_of_two_way
    global number_of_ft
    global number_of_tf
    
    road_type = e['OneWay']
    frm = str(e['FJunction'])
    to = str(e['TJunction'])
    
    if road_type == 'N':
        return
    elif road_type == 'ft':
        if (frm, to) in edges_dict:
            raise Exception(f'Found parallel edge: {frm}, {to}')
        edges_dict[frm, to] = copy.deepcopy(e)
        
        if frm not in adjacency_list:
            adjacency_list[frm] = dict()
        adjacency_list[frm][to] = copy.deepcopy(e)
        
        number_of_ft += 1
    elif road_type == 'tf':
        if (to, frm) in edges_dict:
            raise Exception(f'Found parallel edge: {to}, {frm}')
        edges_dict[to, frm] = copy.deepcopy(e)
        
        if to not in adjacency_list:
            adjacency_list[to] = dict()
        adjacency_list[to][frm] = copy.deepcopy(e)
        
        number_of_tf += 1
    elif road_type == '2':
        if (frm, to) in edges_dict:
            raise Exception(f'Found parallel edge: {frm}, {to}')
        if (to, frm) in edges_dict:
            raise Exception(f'Found parallel edge: {to}, {frm}')
        edges_dict[frm, to] = copy.deepcopy(e)
        edges_dict[to, frm] = copy.deepcopy(e)
        
        if frm not in adjacency_list:
            adjacency_list[frm] = dict()
        adjacency_list[frm][to] = copy.deepcopy(e)
        
        if to not in adjacency_list:
            adjacency_list[to] = dict()
        adjacency_list[to][frm] = copy.deepcopy(e)
        
        number_of_two_way +=2
        
res = shape_edges_df.progress_apply(lambda x: create_edges_dict(x), axis=1)

In [15]:
original_adjacency_list = adjacency_list.copy()

In [51]:
len(adjacency_list)

427358

In [52]:
len(edges_dict)

859377

### Test

In [53]:
# Sanity check
number_of_edges = len(edges_dict)
number_of_ft = shape_edges_df[shape_edges_df['OneWay'] == 'ft'].shape[0]
number_of_tf = shape_edges_df[shape_edges_df['OneWay'] == 'tf'].shape[0]
number_of_two_way = shape_edges_df[shape_edges_df['OneWay'] == '2'].shape[0]
print(number_of_edges)
print(f'number of ft: {number_of_ft}, number of tf: {number_of_tf}, number of two ways * 2: {number_of_two_way*2}')
assert number_of_edges == (number_of_ft + number_of_tf + number_of_two_way*2)

859377
number of ft: 154969, number of tf: 24314, number of two ways * 2: 680094


## Turn Restrictions

In [54]:
turn_restrict_df = gpd.read_file('../../../data/Mapa/Ben Gurion University/Gisrael/restrict.shp')

### Utils

In [55]:
def get_edge_other_node(edge_id, node):
    e = shape_edges_df[shape_edges_df['UserID'] == edge_id]
    
    try:
        f = e['FJunction'].values[0]
        t = e['TJunction'].values[0]
    except:
        print(e['FJunction'])
        print(e['TJunction'])
        raise(Exception())
    
    if f == node:
        return t
    if t == node:
        return f
    
    raise Exception(f'node {node} is not part of the edge {edge_id}')
    
    
############################################################    
######################## Test ##############################
############################################################

# assert get_edge_other_node(1186054, 1058686) == 1058685

In [56]:
nodes = set(list(shape_edges_df['FJunction']) + list(shape_edges_df['TJunction']))
edges = set(shape_edges_df['UserID'])

In [61]:
len(edges)

647504

In [57]:
len(nodes)

489961

### Manipulate Graph according to [Turn Restrictions Algorithm](https://docs.google.com/document/d/1XG5cRIPrrrchI-ub2eU_BYekPyKPA4vKLt3xK62fkS8/edit?usp=sharing)

In [58]:
turn_restrict_df.head(2)

,objid,userid,einterval,junctionid,xinterval,permission,descriptio,geometry
0,1,290963,1186054,1058686,1186052,111111,XXXXXX,None
1,2,333770,1449582,1218598,1449582,111111,XXXXXX,None


In [59]:
skipped_restrictions = []

for i, restrict in turn_restrict_df.iterrows():
    if TEST:
        if (restrict['junctionid'] not in nodes or 
            restrict['einterval'] not in edges or 
            restrict['xinterval'] not in edges):
            continue
    
    u = str(get_edge_other_node(restrict['einterval'], restrict['junctionid']))
    v = str(restrict['junctionid'])
    w = str(get_edge_other_node(restrict['xinterval'], restrict['junctionid']))
    
    original_v = v
    # maybe we already created v_u
    if (u in adjacency_list) and (v not in adjacency_list[u]):
        v = v + '_' + u
    # maybe we already created w_v
    if (v in adjacency_list) and (w not in adjacency_list[v]):
        w = w + '_' + original_v    
    
    # if no edge (u,v) or (v,w) then the restriction is already enforced
    if ((u not in adjacency_list) or (v not in adjacency_list[u]) or 
        (v not in adjacency_list) or (w not in adjacency_list[v])):
        skipped_restrictions.append(restrict)
        continue
        
    if '_' in v:  # v already represents v_u
        # remove (v_u,w)
        del adjacency_list[v][w]
    else:  # v_u was not already created
        # add a new node v_u with a new edge (u, v_u)
        v_u = v + '_' + u
        adjacency_list[v_u] = dict()
        adjacency_list[u][v_u] = adjacency_list[u][v]
    
        # remove (u,v)
        del adjacency_list[u][v]
    
    
        # for each edge (v,x) where x!=w add (v_u, x)
        for x in adjacency_list[v]:
            if x == w:
                continue
            adjacency_list[v_u][x] = adjacency_list[v][x]
        
print(f'Skipped {len(skipped_restrictions)} restrictions')

Skipped 1448 restrictions


In [60]:
len(adjacency_list)

491983

In [22]:
# Verify that all skipped restrictions are blocked roads
count = 0
for row in skipped_restrictions:    
    f_edge_type = shape_edges_df[shape_edges_df['UserID'] == row['einterval']]['OneWay'].values[0] 
    t_edge_type = shape_edges_df[shape_edges_df['UserID'] == row['xinterval']]['OneWay'].values[0] 
    if f_edge_type != 'N' and t_edge_type != 'N':
        count += 1
count

0

In [23]:
# save adjacency list with turn restrictions
with open(OUTPUT_PATH + 'roads_graph_turn_restrictions.pkl','wb') as f:
    pickle.dump(adjacency_list, f)

## Z-Levels
In order to handle z-levels, we must go through all pairs of (u,v),(v,w) edges, and compare T_ZLev(u,v) and F_ZLev(v,w). If they are different, we will mark u->v->w as a turn restriction. 

In [24]:
shape_edges_df.head(2)

,index,UserID,ObjID,FJunction,TJunction,RoadType,OneWay,FromSpeedL,ToSpeedL,AprxSpeedL,Seconds,Length,Roaddirect,DirectionM,F_ZLev,T_ZLev
0,0,1466455,140395,1229728,898653,84,ft,50,50,20,19.573799,36.247675,111111,None,0,0
1,1,1466456,140396,898654,1229728,84,ft,50,50,20,15.640147,28.963154,111111,None,0,0


In [25]:
edges = {'From': [], 'To': [], 'Edge': []}
for f in original_adjacency_list:
    for t in original_adjacency_list[f]:
        edges['From'].append(f)
        edges['To'].append(t)
        edges['Edge'].append(original_adjacency_list[f][t])

edges_df = pd.DataFrame(edges)
    

In [26]:
edges_df.head()

,From,To,Edge
0,1229728,898653,index 0 UserID 1466455 Ob...
1,898654,1229728,index 1 UserID 1466456 Ob...
2,898654,1003088,index 3291 UserID 1090272 Ob...
3,29709,898654,index 2 UserID 944311 Ob...
4,898447,898446,index 3 UserID 943992 Ob...


In [27]:
pairs_of_edges_df = edges_df.merge(edges_df, left_on='To', right_on='From', suffixes=('_F', '_T'))

In [28]:
pairs_of_edges_df.head()

,From_F,To_F,Edge_F,From_T,To_T,Edge_T
0,1229728,898653,index 0 UserID 1466455 Ob...,898653,29686,index 6 UserID 944310 Ob...
1,898654,1229728,index 1 UserID 1466456 Ob...,1229728,898653,index 0 UserID 1466455 Ob...
2,898654,1003088,index 3291 UserID 1090272 Ob...,1003088,29554,index 3288 UserID 1090273 Ob...
3,29709,898654,index 2 UserID 944311 Ob...,898654,1229728,index 1 UserID 1466456 Ob...
4,29709,898654,index 2 UserID 944311 Ob...,898654,1003088,index 3291 UserID 1090272 Ob...


In [29]:
edges_df.shape

(6181, 3)

In [30]:
pairs_of_edges_df.shape

(10136, 6)

In [31]:
z_level_restrictions = {'einterval': [], 'junctionid': [], 'xinterval': []}
for _, row in pairs_of_edges_df.iterrows():    
    middle_node_is_tjunction_in_edge_f = row['To_F'] == str(row['Edge_F']['TJunction'])
    if middle_node_is_tjunction_in_edge_f:
        from_zlev = row['Edge_F']['T_ZLev']
    else:
        assert row['To_F'] == str(row['Edge_F']['FJunction']), f"Edge_F is drawn from {row['Edge_F']['FJunction']} to {row['Edge_F']['TJunction']}, \
                while the current edge goes to {row['To_F']}"
        from_zlev = row['Edge_F']['F_ZLev']
        
    middle_node_is_fjunction_in_edge_t = row['From_T'] == str(row['Edge_T']['FJunction'])
    if middle_node_is_fjunction_in_edge_t:
        to_zlev = row['Edge_T']['F_ZLev']
    else:
        assert row['From_T'] == str(row['Edge_T']['TJunction']), f"Edge_T is drawn from {row['Edge_T']['FJunction']} to {row['Edge_T']['TJunction']}, \
                while the current edge goes from {row['From_F']}"
        to_zlev = row['Edge_T']['T_ZLev']
        
    if from_zlev != to_zlev:
        z_level_restrictions['einterval'].append(int(row['Edge_F']['UserID']))
        z_level_restrictions['junctionid'].append(int(row['To_F']))
        z_level_restrictions['xinterval'].append(int(row['Edge_T']['UserID']))
        
        
z_level_restrictions_df = pd.DataFrame(z_level_restrictions)

In [32]:
z_level_restrictions_df.head(10)

,einterval,junctionid,xinterval
0,1337269,1148239,1337267
1,1337269,1148239,1337268
2,1337268,1148239,1337271
3,1337268,1148239,1337269
4,1337267,1148239,1337271
5,1337267,1148239,1337269
6,1337271,1148239,1337267
7,1337271,1148239,1337268
8,1218913,1077381,1361376
9,1218916,1077381,1218914


In [33]:
skipped_restrictions = []

for i, restrict in z_level_restrictions_df.iterrows():
    u = str(get_edge_other_node(restrict['einterval'], restrict['junctionid']))
    v = str(restrict['junctionid'])
    w = str(get_edge_other_node(restrict['xinterval'], restrict['junctionid']))
    
    original_v = v
    # maybe we already created v_u
    if (u in adjacency_list) and (v not in adjacency_list[u]):
        v = v + '_' + u
    # maybe we already created w_v
    if (v in adjacency_list) and (w not in adjacency_list[v]):
        w = w + '_' + original_v    
    
    # if no edge (u,v) or (v,w) then the restriction is already enforced
    if ((u not in adjacency_list) or (v not in adjacency_list[u]) or 
        (v not in adjacency_list) or (w not in adjacency_list[v])):
        skipped_restrictions.append(restrict)
        continue
        
    if '_' in v:  # v already represents v_u
        # remove (v_u,w)
        del adjacency_list[v][w]
    else:  # v_u was not already created
        # add a new node v_u with a new edge (u, v_u)
        v_u = v + '_' + u
        adjacency_list[v_u] = dict()
        adjacency_list[u][v_u] = adjacency_list[u][v]
    
        # remove (u,v)
        del adjacency_list[u][v]
    
    
        # for each edge (v,x) where x!=w add (v_u, x)
        for x in adjacency_list[v]:
            if x == w:
                continue
            adjacency_list[v_u][x] = adjacency_list[v][x]
        
print(f'Skipped {len(skipped_restrictions)} restrictions')

Skipped 0 restrictions


In [34]:
# save adjacency list with turn restrictions
with open(OUTPUT_PATH + 'roads_graph_turn_restrictions_and_z_levels.pkl','wb') as f:
    pickle.dump(adjacency_list, f)

## Add Weights

In [35]:
weighted_edges = []

for f in adjacency_list:
    for t in adjacency_list[f]:
        weighted_edges.append((f,t,adjacency_list[f][t]['Seconds']))

In [36]:
with open(OUTPUT_PATH + 'roads_graph_weighted_edges.pkl','wb') as f:
    pickle.dump(weighted_edges, f)

## Graph Stats

In [42]:
# Load an existing graph
with open(OUTPUT_PATH + 'roads_graph_weighted_edges.pkl','rb') as f:
    weighted_edges = pickle.load(f)

In [43]:
nodes = [(s, d) for s, d, _ in weighted_edges]
nodes = set([item for sublist in nodes for item in sublist])
print(f'Number of edges: {len(weighted_edges)}\nNumber of nodes: {len(nodes)}')

Number of edges: 949378
Number of nodes: 497721


## Add Nodes Attributes
This is the attribute that should represent the node, such as "number of jobs", "number of residents" etc., to use it in access area / service area computations.
Here, we create a dummy attribute with the value '1' to each node, so for example we count the number of destinations we can reach within time T, rather than number of jobs. 

In [290]:
node_properties = {n : 1 for n in nodes}

with open(OUTPUT_PATH + 'roads_graph_node_attributes.pkl','wb') as f:
    pickle.dump(node_properties, f)

## For Testing: Start Nodes

In [8]:
start_roads_df = gpd.read_file('../../../data/Mapa/test_data/center_tlv/center_tlv.shp')

In [9]:
start_roads_df.head(2)

,street,FromLeft,ToLeft,FromRight,ToRight,StreetCode,ObjID,UserID,FJunction,TJunction,...,Status,RoadFuncti,Flag,Identifier,Roaddirect,CLEARSTREE,REVERSTREE,CLEARCITYN,REVERCITYN,geometry
0,None,0,0,0,0,None,140395,1466455,1229728,898653,...,Operative,Parking,5,0,111111,None,None,úìàáéáéôå,åôé áéáà ìú,"LINESTRING (178742.195 663704.295, 178750.543 ..."
1,çðéåï ìá úì àáéá,0,0,0,0,None,140396,1466456,898654,1229728,...,Operative,Access to Parking,5,0,111111,çðéåïìáúìàáéá,áéáà ìú áì ïåéðç,úìàáéáéôå,åôé áéáà ìú,"LINESTRING (178714.903 663694.601, 178742.195 ..."


In [38]:
with open(OUTPUT_PATH + 'center_tlv_nodes.pkl','wb') as f:
    pickle.dump(list(nodes), f)
